In [1]:
import numpy as np

import sys; sys.path.append('../')

from policies.batch_ens import batch_ens
from utils import load_data
from models.knn_model import KNNModel

from time import time

In [2]:
labels, weights, alpha, nn_ind, sims = load_data('../data/500_nn_data.mat')
model = KNNModel(alpha, weights)

In [4]:
total_budget = 20
batch_size = 4

policy = batch_ens
policy_kwargs = {
    'budget': 4,  # look-ahead: minimum batch_size, larger -> more non-myopic
    'batch_size': batch_size, 'weights': weights,
    'nn_ind': nn_ind, 'sims': sims, 'alpha': alpha,
    'max_n_samples': 16
}

train_ind = np.array([168])
observed_labels = labels[train_ind]
test_ind = np.delete(np.arange(labels.size), train_ind)

for i in range(total_budget // batch_size):
    # Get recommendations and query the labels
    t0 = time()
    chosen_ind = policy(
        train_ind, observed_labels, test_ind, model,
        **policy_kwargs
    )
    duration = time() - t0
    chosen_labels = labels[chosen_ind]
    
    # Update training and test data
    train_ind = np.append(train_ind, chosen_ind)
    observed_labels = np.append(observed_labels, chosen_labels)
    test_ind = np.delete(
        test_ind, np.argwhere(np.isin(test_ind, chosen_ind))
    )
    
    print(f'Iteration {i}:')
    print(f'\tQuery {chosen_ind}')
    print(f'\tLabels {chosen_labels}')
    print(f'\tTook {duration:.2f} seconds')

Iteration 0:
	Query [ 476  210  429 5116]
	Labels [1 1 1 0]
	Took 0.00 seconds
Iteration 1:
	Query [293 169 355 308]
	Labels [1 0 1 1]
	Took 0.01 seconds
Iteration 2:
	Query [296 523 193 462]
	Labels [1 1 1 1]
	Took 0.00 seconds
Iteration 3:
	Query [214 396 202 399]
	Labels [1 1 1 1]
	Took 0.00 seconds
Iteration 4:
	Query [  175   370   310 13441]
	Labels [1 0 0 0]
	Took 0.00 seconds
